In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot
from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input, Dropout, Activation, Dense, MaxPooling2D, Flatten, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adadelta
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
import sklearn
from sklearn.preprocessing import MinMaxScaler

In [ ]:
% ls

drive/  iot23_combined.csv  sample_data/


In [ ]:
filepath = "iot23_combined.csv"
df = pd.read_csv(filepath)

In [ ]:
df

,Unnamed: 0,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
1,1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
2,2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,...,0,0,0,1,0,0,0,0,0,0
3,3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,...,0,0,0,1,0,0,0,0,0,0
4,4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629620,99994,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,0,0,0,1,0,0,0,0,0,0
629621,99995,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,0,0,0,1,0,0,0,0,0,0
629622,99996,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,0,0,0,1,0,0,0,0,0,0
629623,99997,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
del df['Unnamed: 0']

In [ ]:
df['label'].value_counts()

PartOfAHorizontalPortScan    270635
Benign                       158964
Okiru                        149900
DDoS                          39353
C&C                            6787
Attack                         3814
C&C-HeartBeat                   134
C&C-Torii                        30
C&C-FileDownload                  8
Name: label, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df[['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'proto_icmp', 'proto_tcp', 'proto_udp', 'conn_state_OTH', 'conn_state_REJ', 'conn_state_RSTO', 'conn_state_RSTOS0', 'conn_state_RSTR', 'conn_state_RSTRH', 'conn_state_S0', 'conn_state_S1', 'conn_state_S2', 'conn_state_S3', 'conn_state_SF', 'conn_state_SH', 'conn_state_SHR']].values


In [ ]:
X.shape

(629625, 24)

In [ ]:
Y = pd.get_dummies(df['label']).values

In [ ]:
Y.shape

(629625, 9)

In [ ]:
X

array([[2.998796, 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       ...,
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ],
       [0.      , 0.      , 0.      , ..., 0.      , 0.      , 0.      ]])

In [ ]:
df

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,label,proto_icmp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,2.998796,0,0,0.0,3.0,180.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
1,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
2,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,PartOfAHorizontalPortScan,0,...,0,0,0,1,0,0,0,0,0,0
3,2.998804,0,0,0.0,3.0,180.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
4,0.000000,0,0,0.0,1.0,60.0,0.0,0.0,Benign,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629620,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,...,0,0,0,1,0,0,0,0,0,0
629621,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,...,0,0,0,1,0,0,0,0,0,0
629622,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,...,0,0,0,1,0,0,0,0,0,0
629623,0.000000,0,0,0.0,1.0,40.0,0.0,0.0,Okiru,0,...,0,0,0,1,0,0,0,0,0,0


In [ ]:
scaler = MinMaxScaler()

In [ ]:
scaler.fit(X)

MinMaxScaler()

In [ ]:
normalized_x = scaler.transform(X)

In [ ]:
normalized_x

array([[8.16450401e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.04174057e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [2.04174057e-05, 1.32368587e-08, 6.08764998e-06, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [ ]:
normalized_x.shape

(629625, 24)

In [ ]:
scaler.fit(Y)

MinMaxScaler()

In [ ]:
normalized_y = scaler.transform(Y)

In [ ]:
normalized_y

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.]])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(normalized_x, normalized_y, random_state=10, test_size=0.2)

In [ ]:
type(X_train)

numpy.ndarray

In [ ]:
Y_train.shape

(503700, 9)

In [ ]:
model = Sequential()

In [ ]:
model.add(Dense(2000, activation='relu',input_dim=24))
model.add(Dense(1500, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(800,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(400,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(9, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 2000)              50000     
                                                                 
 dense_1 (Dense)             (None, 1500)              3001500   
                                                                 
 dropout (Dropout)           (None, 1500)              0         
                                                                 
 dense_2 (Dense)             (None, 800)               1200800   
                                                                 
 dropout_1 (Dropout)         (None, 800)               0         
                                                                 
 dense_3 (Dense)             (None, 400)               320400    
                                                                 
 dropout_2 (Dropout)         (None, 400)               0

In [ ]:
import time

In [ ]:
start = time.time()
print('program start...')
print()

history = model.fit(X_train, Y_train, epochs = 10, batch_size=256, validation_data=(X_test,Y_test),verbose=1)

print()
end = time.time()
print('program end...')
print()
print('time cost: ')
print(end - start, 'seconds')

program start...

Epoch 1/10
1968/1968 [==============================] - 708s 358ms/step - loss: 1.1203 - accuracy: 0.5275 - val_loss: 1.1039 - val_accuracy: 0.5307
Epoch 2/10
1968/1968 [==============================] - 702s 357ms/step - loss: 1.1057 - accuracy: 0.5313 - val_loss: 1.1023 - val_accuracy: 0.5307
Epoch 3/10
1968/1968 [==============================] - 709s 360ms/step - loss: 1.1050 - accuracy: 0.5311 - val_loss: 1.1017 - val_accuracy: 0.5305
Epoch 4/10
1968/1968 [==============================] - 711s 361ms/step - loss: 1.1048 - accuracy: 0.5311 - val_loss: 1.1021 - val_accuracy: 0.5305
Epoch 5/10
1968/1968 [==============================] - 711s 361ms/step - loss: 1.1045 - accuracy: 0.5311 - val_loss: 1.1023 - val_accuracy: 0.5305
Epoch 6/10
1968/1968 [==============================] - 711s 361ms/step - loss: 1.1041 - accuracy: 0.5311 - val_loss: 1.1020 - val_accuracy: 0.5305
Epoch 7/10
1968/1968 [==============================] - 715s 363ms/step - loss: 1.1043 - accur